In [1]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 4.3 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 a0529.csv		       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
dataframe_path_530 = 'dataframe_csv/dataframe0529.csv'

In [8]:
dataframe_530 = pd.read_csv(dataframe_path_530)

In [9]:
dataframe_530=pd.DataFrame(dataframe_530,columns=['0','1'])

In [10]:
dataframe_530

,0,1
0,minneapolis,16071
1,minnesota,4009
2,america,1044
3,st. paul,919
4,hennepin county,761
...,...,...
1554,river | savannah,1
1555,the city failed us',1
1556,new yorker)background,1
1557,black community,1


In [11]:
dataframe_530.shape[1]

2

In [12]:
columns2=['City','Number']

In [13]:
#len(columns1)
len(columns2)

2

In [14]:
dataframe_530.columns=columns2

In [15]:
dataframe_530

,City,Number
0,minneapolis,16071
1,minnesota,4009
2,america,1044
3,st. paul,919
4,hennepin county,761
...,...,...
1554,river | savannah,1
1555,the city failed us',1
1556,new yorker)background,1
1557,black community,1


In [16]:
list_city=dataframe_530['City'].tolist()
list_num=dataframe_530['Number'].tolist()
list_lat=[]
list_lon=[]
list_address=[]

In [17]:
len(list_city)

1559

In [18]:
del_list=[]

In [19]:
from geopy.exc import GeocoderTimedOut
counter=0
for i in list_city:
  #search_string=dataframe_529['City'][i]
  geolocator = Nominatim(user_agent="address_development")
  try:
    location = geolocator.geocode(i,timeout=30)
    if(location==None):
      del_list.append(counter)
      time.sleep(1)
    else:
      list_lat.append(location.latitude)
      list_lon.append(location.longitude)
      list_address.append(location.address)
      time.sleep(1)
    counter+=1
  except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(Location, e.msg))

In [20]:
len(del_list)

218

In [21]:
len(dataframe_530)

1559

In [22]:
for i in del_list:
  dataframe_530=dataframe_530.drop(labels=i,axis=0)

In [23]:
len(dataframe_530)

1341

In [24]:
dataframe_530.reset_index(drop=True, inplace=True)

In [25]:
list_city=dataframe_530['City'].tolist()
list_num=dataframe_530['Number'].tolist()

In [26]:
print(counter)

1559


In [27]:
dict={"lat" : list_lat,
   "lon" : list_lon,
   "address" : list_address}
#dataframe_a1=DataFrame(dict)
#dataframe_a1

In [28]:
from pandas.core.frame import DataFrame
dataframe_a1=DataFrame(dict)
dataframe_a1

,lat,lon,address
0,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,45.989659,-94.611329,"Minnesota, United States"
2,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
3,44.949749,-93.093103,"Saint Paul, Ramsey County, Minnesota, United S..."
4,45.025723,-93.486505,"Hennepin County, Minnesota, United States"
...,...,...,...
1336,42.341765,-70.966160,"Boston Harbor, Boston, Suffolk County, Massach..."
1337,51.453802,-2.597298,"Bristol, City of Bristol, South West England, ..."
1338,30.739633,-89.490895,"Savannah, Pearl River County, Mississippi, Uni..."
1339,43.262258,-2.927797,"Black, Berastegui kalea, Abando, Bilbao, Bizka..."


In [29]:
dataframe_530

,City,Number
0,minneapolis,16071
1,minnesota,4009
2,america,1044
3,st. paul,919
4,hennepin county,761
...,...,...
1336,boston harbor,1
1337,bristol,1
1338,river | savannah,1
1339,black community,1


In [30]:
len(list_lat)

1341

In [31]:
dataframe_result=pd.concat([dataframe_530,dataframe_a1],axis=1)
dataframe_result

,City,Number,lat,lon,address
0,minneapolis,16071,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,minnesota,4009,45.989659,-94.611329,"Minnesota, United States"
2,america,1044,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
3,st. paul,919,44.949749,-93.093103,"Saint Paul, Ramsey County, Minnesota, United S..."
4,hennepin county,761,45.025723,-93.486505,"Hennepin County, Minnesota, United States"
...,...,...,...,...,...
1336,boston harbor,1,42.341765,-70.966160,"Boston Harbor, Boston, Suffolk County, Massach..."
1337,bristol,1,51.453802,-2.597298,"Bristol, City of Bristol, South West England, ..."
1338,river | savannah,1,30.739633,-89.490895,"Savannah, Pearl River County, Mississippi, Uni..."
1339,black community,1,43.262258,-2.927797,"Black, Berastegui kalea, Abando, Bilbao, Bizka..."


In [32]:
pd.set_option('display.max_rows', None)

In [33]:
list_citya=dataframe_result['City'].tolist()
list_numa=dataframe_result['Number'].tolist()
list_lata=dataframe_result['lat'].tolist()
list_lona=dataframe_result['lon'].tolist()
list_addressa=dataframe_result['address'].tolist()

In [34]:
for i in range(len(list_citya)):
  j=i+1
  while(j<len(list_citya)):
    if(list_lata[i]==list_lata[j]):
      if(list_lona[i]==list_lona[j]):
        list_numa[i]+=list_numa[j]
        del list_citya[j]
        del list_numa[j]
        del list_lata[j]
        del list_lona[j]
        del list_addressa[j]
        continue
    j+=1


In [35]:
dataframe_result

,City,Number,lat,lon,address
0,minneapolis,16071,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,minnesota,4009,45.989659,-94.611329,"Minnesota, United States"
2,america,1044,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
3,st. paul,919,44.949749,-93.093103,"Saint Paul, Ramsey County, Minnesota, United S..."
4,hennepin county,761,45.025723,-93.486505,"Hennepin County, Minnesota, United States"
5,the united states,531,29.633008,-97.988594,"The Summit, Guadalupe County, Texas, United St..."
6,denver,515,39.739236,-104.984862,"Denver, Denver County, Colorado, United States"
7,u.s.,492,48.630579,9.417545,"Unterer See, Kirchheim unter Teck, Vereinbarte..."
8,us,463,39.783730,-100.445882,United States
9,houston,379,29.758938,-95.367697,"Houston, Harris County, Texas, United States"


In [36]:
dict_result_a={"City" : list_citya,
   "Number" : list_numa,
   "lat" : list_lata,
   "lon" : list_lona,
   "address" : list_addressa}
from pandas.core.frame import DataFrame
dataframe_result_a1=DataFrame(dict_result_a)
dataframe_result_a1

,City,Number,lat,lon,address
0,minneapolis,16073,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,minnesota,4365,45.989659,-94.611329,"Minnesota, United States"
2,america,1044,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
3,st. paul,930,44.949749,-93.093103,"Saint Paul, Ramsey County, Minnesota, United S..."
4,hennepin county,765,45.025723,-93.486505,"Hennepin County, Minnesota, United States"
5,the united states,553,29.633008,-97.988594,"The Summit, Guadalupe County, Texas, United St..."
6,denver,515,39.739236,-104.984862,"Denver, Denver County, Colorado, United States"
7,u.s.,664,48.630579,9.417545,"Unterer See, Kirchheim unter Teck, Vereinbarte..."
8,us,524,39.783730,-100.445882,United States
9,houston,379,29.758938,-95.367697,"Houston, Harris County, Texas, United States"


In [37]:
dataframe_result_a1.to_csv('dataframe_result_0529.csv')

In [38]:
dataframe_result_a1

,City,Number,lat,lon,address
0,minneapolis,16073,44.977300,-93.265469,"Minneapolis, Hennepin County, Minnesota, 55415..."
1,minnesota,4365,45.989659,-94.611329,"Minnesota, United States"
2,america,1044,51.437148,5.979900,"America, Horst aan de Maas, Limburg, Nederland"
3,st. paul,930,44.949749,-93.093103,"Saint Paul, Ramsey County, Minnesota, United S..."
4,hennepin county,765,45.025723,-93.486505,"Hennepin County, Minnesota, United States"
5,the united states,553,29.633008,-97.988594,"The Summit, Guadalupe County, Texas, United St..."
6,denver,515,39.739236,-104.984862,"Denver, Denver County, Colorado, United States"
7,u.s.,664,48.630579,9.417545,"Unterer See, Kirchheim unter Teck, Vereinbarte..."
8,us,524,39.783730,-100.445882,United States
9,houston,379,29.758938,-95.367697,"Houston, Harris County, Texas, United States"


In [39]:
geolocator = Nominatim(user_agent="address_development")
location = geolocator.geocode('the united states	')

print(f'Address Match: {location.address}')
print(f'Coordinates: {(location.latitude, location.longitude)}')

Address Match: The Summit, Guadalupe County, Texas, United States
Coordinates: (29.6330078, -97.98859404468138)
